In [1]:
import connectorx as cx
import pandas as pd
import numpy as np
import os
import duckdb
from datetime import datetime
import glob
import multiprocessing as mp
from functools import partial
import time

In [2]:
def calculate_daily_factor(trading_date):
    import pandas as pd
    import duckdb
    import os
    
    date_str = pd.to_datetime(trading_date).strftime('%Y%m%d')
    order_pth = f"/data/cephfs/order/{date_str}.parquet"
    
    if not os.path.exists(order_pth):
        raise FileNotFoundError(f"数据文件不存在: {order_pth}")

    # 为每个进程创建独立的duckdb连接
    conn = duckdb.connect(database=':memory:')
    
    # 初始化结果DataFrame
    result_df = None
    
    # 跟踪创建的临时表，以便在finally块中清理
    created_tables = []
    
    try:
        # 买方订单
        bid_query = f"""
        SELECT 
            security_code,
            order_side, 
            order_type,
            order_details,
            order_price,
            order_price_adj,
            order_volume,
            order_time,
            DATE_TRUNC('minute', 
            ('{trading_date}'::DATE + MAKE_TIME(
                    FLOOR(order_time / 10000000)::int,
                    (FLOOR(order_time / 100000) % 100)::int,
                    (FLOOR(order_time / 1000) % 100)::numeric + 
                    (order_time % 1000)::numeric / 1000) 
            )::TIMESTAMP
                ) AS order_minute
        FROM '{order_pth}'
        WHERE order_type = 'A'
            AND order_details = 'L'
            AND order_side = 1
        """
        
        # 卖方订单
        ask_query = f"""
        SELECT 
            security_code,
            order_side, 
            order_type,
            order_details,
            order_price,
            order_price_adj,
            order_volume,
            order_time,
            DATE_TRUNC('minute', 
            ('{trading_date}'::DATE + MAKE_TIME(
                    FLOOR(order_time / 10000000)::int,
                    (FLOOR(order_time / 100000) % 100)::int,
                    (FLOOR(order_time / 1000) % 100)::numeric + 
                    (order_time % 1000)::numeric / 1000) 
            )::TIMESTAMP
                ) AS order_minute
        FROM '{order_pth}'
        WHERE order_type = 'A'
            AND order_details = 'L'
            AND order_side = -1
        """
            
        conn.execute(f"CREATE TEMPORARY TABLE bid_orders AS {bid_query}")
        created_tables.append('bid_orders')
        
        conn.execute(f"CREATE TEMPORARY TABLE ask_orders AS {ask_query}")
        created_tables.append('ask_orders')

        sides = ['bid','ask']

        for side in sides:
            factor_name = f"{side}_price_divergence"

            # 计算价格分位数
            percentile_query = f"""
            CREATE TEMPORARY TABLE {side}_percentiles AS
            SELECT 
                security_code,
                order_minute,
                PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY order_price) AS p01,
                PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY order_price) AS p99
            FROM {side}_orders
            GROUP BY security_code, order_minute
            """
            conn.execute(percentile_query)
            created_tables.append(f'{side}_percentiles')
            
            # P01-P99缩尾表
            winsorized_query_p01_p99 = f"""
            CREATE TEMPORARY TABLE {side}_p01_p99 AS
            SELECT 
                b.security_code,
                b.order_side,
                CASE 
                    WHEN b.order_price < p.p01 THEN p.p01
                    WHEN b.order_price > p.p99 THEN p.p99
                    ELSE b.order_price
                END AS order_price,
                b.order_volume,
                b.order_time,
                b.order_minute
            FROM {side}_orders b
            JOIN {side}_percentiles p ON b.security_code = p.security_code AND b.order_minute = p.order_minute
            """
            conn.execute(winsorized_query_p01_p99)
            created_tables.append(f'{side}_p01_p99')

            # 计算订单加权平均对数价格
            factor_query = f"""
            WITH {side}_info AS (
                SELECT 
                    security_code,
                    order_minute,
                    order_volume,
                    order_price
                FROM {side}_p01_p99
                WHERE order_price > 0  -- 确保价格为正
            ),
            {side}_stats AS (
                SELECT 
                    security_code,
                    order_minute,
                    SUM(order_volume) AS total_volume,
                    CASE WHEN SUM(order_volume) > 0 
                        THEN SUM(LN(order_price) * order_volume) / SUM(order_volume)
                        ELSE NULL 
                    END AS weighted_log_price
                FROM {side}_info
                GROUP BY security_code, order_minute
            ),
            {side}_price_divergence AS (
                SELECT 
                    o.security_code,
                    o.order_minute,
                    SQRT(
                        CASE WHEN ANY_VALUE(s.total_volume) > 0
                            THEN SUM(POWER(LN(o.order_price) - s.weighted_log_price, 2) * o.order_volume) / ANY_VALUE(s.total_volume)
                            ELSE 0 
                        END
                    ) AS {factor_name}_p01p99
                FROM {side}_info o
                JOIN {side}_stats s ON o.security_code = s.security_code AND o.order_minute = s.order_minute
                WHERE s.weighted_log_price IS NOT NULL
                GROUP BY o.security_code, o.order_minute
            )
            SELECT 
                security_code,
                order_minute,
                {factor_name}_p01p99
            FROM {side}_price_divergence
            """
            
            current_df = conn.execute(factor_query).fetchdf()
            
            if result_df is None:
                result_df = current_df
            else:
                result_df = result_df.merge(
                    current_df,
                    on=['security_code','order_minute'],
                    how='outer'
                )
    finally:
        # 清理临时表
        for table in created_tables:
            try:
                conn.execute(f"DROP TABLE IF EXISTS {table}")
            except:
                pass
        conn.close()
    
    # 如果没有结果，返回空DataFrame
    if result_df is None or result_df.empty:
        columns = ['security_code', 'order_minute', 'bid_price_divergence_p01p99', 'ask_price_divergence_p01p99']
        return pd.DataFrame(columns=columns)
    
    return result_df


In [3]:
def calculate_daily_rolling_average(min_window=5):
    import glob
    import os
    import pandas as pd
    import concurrent.futures
    from functools import partial
    
    # 源数据目录
    data_dir = "./factors/percent_minute_price_divergence"
    # 日度统计结果保存目录
    daily_stats_dir = "./factors/percent_minute_price_divergence/DailyMinute"

    os.makedirs(daily_stats_dir, exist_ok=True)
    
    # 获取所有日期的parquet文件
    parquet_files = sorted(glob.glob(os.path.join(data_dir, "*.parquet")))
    
    if not parquet_files:
        raise ValueError("未找到分钟级数据文件")
    
    print(f"在目录 {data_dir} 中找到 {len(parquet_files)} 个Parquet文件")
    
    # 定义处理单个日期文件的函数
    def process_daily_file(file_path):
        try:
            # 读取单日数据
            df = pd.read_parquet(file_path)
            
            # 提取日期信息（从文件名）
            date_str = os.path.basename(file_path).split('.')[0]
            
            # 计算每个股票的平均值和标准差
            daily_bid=df.groupby(['security_code'])['bid_price_divergence_p01p99'].mean().reset_index()
            daily_ask= df.groupby(['security_code'])['ask_price_divergence_p01p99'].mean().reset_index()

            daily_bid.columns = ['security_code','bid_price_divergence_p01p99']
            daily_ask.columns = ['security_code','ask_price_divergence_p01p99']
            
            # 合并平均值和标准差
            daily_factor = pd.merge(
                daily_bid,
                daily_ask,
                on='security_code',
                how='outer'
            )
            
            # 添加日期列
            daily_factor['date'] = date_str
            
            # 保存为新的parquet文件
            output_file = os.path.join(daily_stats_dir, f"{date_str}.parquet")
            daily_factor.to_parquet(output_file)
            
            print(f"处理完成: {date_str}, 包含 {len(daily_factor)} 条记录")
            return output_file
        except Exception as e:
            print(f"处理文件 {file_path} 时出错: {str(e)}")
            return None
    
    # 第一阶段：并行处理每个日期文件
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        processed_files = list(executor.map(process_daily_file, parquet_files))
    
    # 过滤掉处理失败的文件
    processed_files = [f for f in processed_files if f is not None]
    print(f"成功处理 {len(processed_files)} 个日期文件")
    
    # 第二阶段：读取所有日度统计文件并合并
    daily_stats_files = sorted(glob.glob(os.path.join(daily_stats_dir, "*.parquet")))
    
    if not daily_stats_files:
        raise ValueError("未找到日度统计文件")
    
    # 读取并合并所有日度统计文件
    all_daily_stats = []
    for file in daily_stats_files:
        df = pd.read_parquet(file)
        all_daily_stats.append(df)
    
    # 合并所有日期的数据
    all_stats = pd.concat(all_daily_stats, ignore_index=True)
    
    # 确保日期格式正确并排序
    all_stats['date'] = pd.to_datetime(all_stats['date'], format='%Y%m%d')
    all_stats = all_stats.sort_values(['date', 'security_code'])
    
    # 获取所有个股代码
    all_securities = all_stats['security_code'].unique()
    print(f"共有 {len(all_securities)} 个不同的证券代码")
    
    # 最终输出目录
    output_path = "/data/home/lexuanchen/Factors/Order/Signal/Percent_Minute_Price_Divergence"
    os.makedirs(output_path, exist_ok=True)
    
    factor_names = ['bid_price_divergence_p01p99','ask_price_divergence_p01p99']
    
    # 定义处理每个因子的函数
    def process_factor(factor_name, all_stats, min_window):
        # 转置成宽表进行滚动计算
        pivot_data = all_stats.pivot(index='date', columns='security_code', values=factor_name)
        
        # 对宽表直接应用rolling
        rolling_avg = pivot_data.rolling(window=20,min_periods=min_window).mean()
        
        # 将结果转换回长格式
        factor_df = rolling_avg.stack().reset_index()
        factor_df.columns = ['date', 'security_code', f'{factor_name}_rolling']
        
        # 计算每日因子覆盖率
        non_nan_counts = factor_df.dropna(subset=[f'{factor_name}_rolling']).groupby('date').size()
        
        # 计算覆盖率
        coverage = non_nan_counts / len(all_securities)
        print(f"\n{factor_name}因子覆盖率统计: 平均={coverage.mean():.2f}, 最小={coverage.min():.2f}")
        

        factor_df['date'] = factor_df['date'].dt.strftime('%Y%m%d')
        
        # 保存结果
        output_file_path = f"{output_path}/{factor_name}.csv"
        factor_df.to_csv(output_file_path, index=False)
        
        return output_file_path
    
    # 使用多线程处理每个因子
    with concurrent.futures.ThreadPoolExecutor(max_workers=len(factor_names)) as executor:
        process_func = partial(process_factor, all_stats=all_stats, min_window=min_window)
        result_files = list(executor.map(process_func, factor_names))
    
    print(f"所有因子处理完成，结果保存在: {output_path}")
    return output_path


In [ ]:
# 获取Raw factor

daily_stats_dir = "./factors/percent_minute_price_divergence/DailyMinute"
daily_stats_files = sorted(glob.glob(os.path.join(daily_stats_dir, "*.parquet")))
    
if not daily_stats_files:
    raise ValueError("未找到日度统计文件")

# 读取并合并所有日度统计文件
all_daily_stats = []
for file in daily_stats_files:
    df = pd.read_parquet(file)
    all_daily_stats.append(df)

# 合并所有日期的数据
all_stats = pd.concat(all_daily_stats, ignore_index=True)

# 确保日期格式正确并排序
all_stats['date'] = pd.to_datetime(all_stats['date'], format='%Y%m%d')
all_stats = all_stats.sort_values(['date', 'security_code'])

# 获取所有个股代码
all_securities = all_stats['security_code'].unique()
print(f"共有 {len(all_securities)} 个不同的证券代码")

# 最终输出目录
output_path = "/data/home/lexuanchen/Factors/Order/Signal/Percent_Minute_Price_Divergence_2"

os.makedirs(output_path,exist_ok=True)

factor_names = ['bid_price_divergence_p01p99','ask_price_divergence_p01p99']

for factor in factor_names:

    df = all_stats[['date','security_code', factor]]

    df.to_csv(f"{output_path}/Raw_{factor}.csv")

    print(f"{factor} 原始因子储存完毕")

共有 5348 个不同的证券代码
bid_price_divergence_p01p99 原始因子储存完毕
ask_price_divergence_p01p99 原始因子储存完毕


In [4]:
def process_single_date(trading_date, factor_name):
    
    """处理单个交易日的函数"""
   
    date_str = trading_date.strftime('%Y%m%d')
    output_dir = f"./factors/{factor_name}"
    file_path = f"{output_dir}/{date_str}.parquet"
    
    # 如果文件已存在，跳过处理
    if os.path.exists(file_path):
        print(f"文件已存在，跳过: {file_path}")
        return date_str, True, "已存在"
    
    # 检查原始数据文件是否存在
    data_file = f"/data/cephfs/order/{date_str}.parquet"
    if not os.path.exists(data_file):
        print(f"跳过日期 {date_str}: 原始数据文件不存在")
        return date_str, False, "数据文件不存在"
    
    try:
        daily_factor_df = calculate_daily_factor(trading_date)
        
        daily_factor_df['date'] = date_str

        daily_factor_df.to_parquet(file_path, index=False)
        print(f"已生成因子文件: {file_path}")
        return date_str, True, "成功"
        
    except Exception as e:
        error_msg = str(e)
        print(f"处理日期 {date_str} 时出错: {error_msg}")
        return date_str, False, error_msg


def derive_daily_factor(start_date, end_date, factor_name, num_processes=30):
    start_time = time.time()
    
    trading_dates = pd.date_range(start=start_date, end=end_date, freq='D')
    
    # 创建输出目录
    output_dir = f"./factors/{factor_name}"
    os.makedirs(output_dir, exist_ok=True)
    
    # 创建进程池
    pool = mp.Pool(processes=num_processes)
    
    # 创建带有固定参数的处理函数
    process_date_with_args = partial(process_single_date, factor_name=factor_name)
    
    # 提交所有任务到进程池并获取结果
    print(f"开始使用 {num_processes} 个进程并行处理 {len(trading_dates)} 个交易日...")
    results = pool.map(process_date_with_args, trading_dates)
    
    # 关闭进程池
    pool.close()
    pool.join()
    
    # 分析处理结果
    processed_dates = [date_str for date_str, success, _ in results if success]
    skipped_dates = [date_str for date_str, success, _ in results if not success]
    
    # 打印处理结果摘要
    print(f"\n处理完成:")
    print(f"成功处理 {len(processed_dates)} 个日期")
    print(f"跳过 {len(skipped_dates)} 个日期")

    return output_dir


In [ ]:
def main():

    start_date = '2016-06-20'
    end_date = '2019-01-05'
    
    all_factors = ['bid_price_divergence_p01p99','ask_price_divergence_p01p99']

    factor_name = "percent_minute_price_divergence"

    factor_dir = derive_daily_factor (start_date, end_date, factor_name)
    print(f"所有因子文件已生成在目录: {factor_dir}")

    result_df_path =  calculate_daily_rolling_average(min_window=5)
    
    # # 保存结果为CSV
    # output_path = "/data/home/lexuanchen/Factors/Order/Signal/Percent_Minute_Price_Divergence"
    # os.makedirs(output_path, exist_ok=True)

    # print(f"共计 {len(result_df)} 条记录")

    # print("\n数据预览:")
    # print(result_df.head())


    # for factor in all_factors:
    #     output_file_path = f"{output_path}/{factor}.csv"
    #     factor_df = result_df[['date', 'security_code', factor]]
    #     factor_df.to_csv(output_file_path, index=False)


# 执行主函数
if __name__ == "__main__":
    main()


跳过日期 20160807: 原始数据文件不存在跳过日期 20160730: 原始数据文件不存在跳过日期 20160916: 原始数据文件不存在跳过日期 20161002: 原始数据文件不存在
跳过日期 20160924: 原始数据文件不存在跳过日期 20170114: 原始数据文件不存在
跳过日期 20170122: 原始数据文件不存在

跳过日期 20160917: 原始数据文件不存在

跳过日期 20161127: 原始数据文件不存在跳过日期 20170130: 原始数据文件不存在跳过日期 20170115: 原始数据文件不存在

跳过日期 20161119: 原始数据文件不存在跳过日期 20160925: 原始数据文件不存在
跳过日期 20161003: 原始数据文件不存在


跳过日期 20160731: 原始数据文件不存在跳过日期 20161004: 原始数据文件不存在



跳过日期 20160918: 原始数据文件不存在跳过日期 20161120: 原始数据文件不存在

跳过日期 20170131: 原始数据文件不存在跳过日期 20161005: 原始数据文件不存在

跳过日期 20170201: 原始数据文件不存在
跳过日期 20161006: 原始数据文件不存在
跳过日期 20170202: 原始数据文件不存在跳过日期 20161007: 原始数据文件不存在

跳过日期 20161008: 原始数据文件不存在
跳过日期 20161009: 原始数据文件不存在
开始使用 30 个进程并行处理 930 个交易日...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
import pandas as pd

ask_df = pd.read_csv("/data/home/lexuanchen/Factors/Order/Signal/Minute_Price_Divergence/ask_price_divergence.csv")
bid_df = pd.read_csv("/data/home/lexuanchen/Factors/Order/Signal/Minute_Price_Divergence/bid_price_divergence.csv")

ask_df['ask_price_divergence_rolling'] = -ask_df['ask_price_divergence_rolling']
bid_df['bid_price_divergence_rolling'] = -bid_df['bid_price_divergence_rolling']

bid_df.to_csv('/data/home/lexuanchen/Factors/Order/Signal/Minute_Price_Divergence/bid_price_divergence.csv',index=False)
ask_df.to_csv('/data/home/lexuanchen/Factors/Order/Signal/Minute_Price_Divergence/ask_price_divergence.csv',index=False)
